In [ ]:
import numpy as np
import pandas as pd 
import altair as alt
import re 
from commons import data_processing
from commons import common_objects as co
import matplotlib.pyplot as plt
from venn import venn
from scipy import stats, signal
from matplotlib_venn import venn2
import warnings

warnings.filterwarnings('ignore')
alt.data_transformers.disable_max_rows()

In [ ]:
pgc = data_processing.get_files(r'./data/DIA', exts=['PGC.tsv'])[0]
rplc = data_processing.get_files(r'./data/DIA/', exts=['RPLC.tsv'])[0]

column_colors = alt.Color('column:N',
    scale=alt.Scale(
        domain=["PGC", "RPLC"],
        range=["#D56E3B", "#58728C"]
    ))


In [ ]:
files = [rplc, pgc]
columns = ['RPLC', 'PGC']

df = pd.DataFrame()

for (col, file) in zip(columns, files):
    print(col, file)
    _df = pd.read_csv(file, delimiter='\t')
    _df.loc[:, 'column'] = col 
    _df.loc[:, 'sample'] = _df.Run.map(lambda x: x.split('_')[-3])
    _df.loc[:, 'tech_rep'] = _df.Run.map(lambda x: x.split('_')[-1])

    df = pd.concat([df, _df])
    df.reset_index(inplace=True, drop=True)

In [ ]:
g = df.groupby(['sample', 'column', 'Modified.Sequence']).mean()
g = g.reset_index()

total = alt.Chart(g).mark_area(
    opacity=0.6
).transform_density(
    'RT',
    as_=['RT', 'density'],
    groupby=['sample', 'column']
).encode(
    x=alt.X('RT:Q', title='Retention Time',
        axis=co.alt_axis),
    y=alt.Y('density:Q', title='',
        axis=co.alt_axis),
    color=column_colors,
    row='sample:N'
).properties(
    height=150,
    width=400,
    title='Total'
)

unique = alt.Chart(g.drop_duplicates('Modified.Sequence',keep=False)).mark_area(
    opacity=0.6
).transform_density(
    'RT',
    as_=['RT', 'density'],
    groupby=['sample', 'column']
).encode(
    x=alt.X('RT:Q', title='Retention Time',
        axis=co.alt_axis),
    y=alt.Y('density:Q', title='',
        axis=co.alt_axis),
    color=column_colors,
    row='sample:N'
).properties(
    height=150,
    width=400,
    title='Unique'
)

# total | unique

In [ ]:
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor

def pour(seq, scale='gravy'):
    desc = PeptideDescriptor(seq, scale)
    desc.calculate_global()
    return desc.descriptor[0][0]


def accessory(seq, scale='polarity'):
    desc = PeptideDescriptor(seq, scale)
    desc.calculate_global()
    return desc.descriptor[0][0]

def aa_dict(composition):
    f = {}
    for el in composition.split():
        f[el[0]] = f.get(el[0], 0) + int(el[1:])
    return f

def get_sequence(seq):
    desc = GlobalDescriptor(seq)
    desc.formula()
    form = desc.descriptor[0][0]
    return aa_dict(form)

def get_global(seq):
    desc = GlobalDescriptor(seq) 
    desc.calculate_all(ph=1.0)
    res = desc.descriptor[0]
    cats = ['Length', 'MW', 'Charge', 'ChargeDensity', 'pI', 'InstabilityInd', 'Aromaticity', 'AliphaticInd', 'BomanInd', 'HydrophRatio']
    ret = dict(zip(cats, res))
    return ret



In [ ]:
time = df.groupby(['column', 'Modified.Sequence', 'Stripped.Sequence']).mean()
time = time.reset_index()

# time.loc[:, 'length'] = time['Stripped.Sequence'].map(lambda x: len(x))
# time.loc[:, 'gravy'] = time['Stripped.Sequence'].map(pour)
# time.loc[:, 'polarity'] = time['Stripped.Sequence'].map(accessory)


counts = time['Modified.Sequence'].value_counts()
valid = counts[counts.values==2].keys()
time = time[time['Modified.Sequence'].isin(valid)]


time_diff = pd.DataFrame()

for seq, s_frame in data_processing.iterate_contents('Modified.Sequence', time, get_item=True):
    stripped = s_frame['Stripped.Sequence'].values[0]
    t_diff = s_frame[['Modified.Sequence', 'RT']].groupby('Modified.Sequence').diff(periods=-1).values[0]
    # display(s_frame[['Modified.Sequence', 'column', 'RT']])
    
    s = pd.DataFrame({
        'mod_sequence':seq,
        'sequence':stripped,
        'rt_diff':t_diff
    })    

    time_diff = pd.concat([time_diff, s]).reset_index(drop=True)

time_diff.loc[:, 'length'] = time_diff['sequence'].map(lambda x: len(x))
time_diff.loc[:, 'gravy'] = time_diff['sequence'].map(pour)
time_diff.loc[:, 'polarity'] = time_diff['sequence'].map(accessory)



In [ ]:
compositions = pd.DataFrame()

for peptide in time_diff.sequence:
    aa_comp = pd.DataFrame(get_sequence(peptide), index=range(1))
    chars = pd.DataFrame(get_global(peptide), index=range(1))
    d = pd.concat([aa_comp, chars], axis=1)
    d.loc[:, 'sequence'] = peptide

    compositions = pd.concat([compositions, d]).reset_index(drop=True)

compositions


In [ ]:
character_merge = time_diff.merge(compositions, on='sequence')
import seaborn as sns 
sns.heatmap(character_merge.iloc[:, 2:].corr())

In [ ]:
def accessory(seq, scale='refractivity'):
    desc = PeptideDescriptor(seq, scale)
    desc.calculate_global()
    return desc.descriptor[0][0]

# time_diff.loc[:, 'accessory'] = time_diff.sequence.map(accessory)
time_diff.loc[:, 'len_penalty'] = (time_diff.length - time_diff.length.mean())
time_diff.loc[:, 'accessory'] = time_diff.gravy*-1**(1/(1-time_diff.polarity))

base = alt.Chart(time_diff).encode(
    x=alt.X('rt_diff:Q',
        bin=alt.Bin(
            step=1
        ),
        axis=co.alt_axis),
    y=alt.Y('mean(polarity):Q',
        axis=co.alt_axis),
).transform_calculate(
    composite=alt.datum.gravy*-1 * alt.datum.polarity
)

# band = alt.Chart(time_diff).encode(
#     x=alt.X('rt_diff:Q',
#         bin=alt.Bin(step=2)),
#     y=alt.Y('composite:Q')
# ).transform_calculate(
#     composite=alt.datum.gravy*-1 * alt.datum.polarity
# )

line = base.mark_line(size=1,interpolate='basis')
band = base.mark_errorband(extent='ci', interpolate='basis')
line + band

In [ ]:
counts = g['Modified.Sequence'].value_counts()
valid = counts[counts.values==4].keys()
g_shared = g[g['Modified.Sequence'].isin(valid)]


alt.Chart(g_shared).mark_area(
    opacity=0.6
).transform_density(
    'RT',
    as_=['RT', 'density'],
    groupby=['sample', 'column']
).encode(
    x=alt.X('RT:Q', title='Retention Time'),
    y=alt.Y('density:Q', title=''),
    color=column_colors,
    row='column:N'
)

In [ ]:
g = df.groupby(['sample', 'column', 'Modified.Sequence']).mean()

samples = df['sample'].unique()
columns = df['column'].unique()

fig, axs = plt.subplots(1, 2, figsize=(10,5))
i = 0
for s in samples:
    overlap = {}
    for c in columns:
        small = g.loc[(s, c), :]
        small.reset_index(inplace=True)
        overlap[c] = set(small["Modified.Sequence"].tolist())
    venn(overlap, ax=axs[i])
    axs[i].set_title(s)

    i += 1

In [ ]:
d = df[(df["Proteotypic"]==1)&(df["Protein.Q.Value"]<=0.01)]
d = d.groupby(['sample', 'column', 'Protein.Ids']).mean()

samples = df['sample'].unique()

fig, axs = plt.subplots(1, 2)
i = 0
for s in samples:
    overlap = {}
    for c in columns:
        small = d.loc[(s, c), :]
        small.reset_index(inplace=True)
        overlap[c] = set(small["Protein.Ids"].tolist())
    venn(overlap, ax=axs[i])
    axs[i].set_title(s)

    i += 1

In [ ]:
alt.Chart(d.reset_index()).mark_area().encode(
    x=alt.X('RT:Q', title='RT',
        bin=alt.Bin(
            step=5
        )),
    y=alt.Y('count():Q', title=''),
    color=alt.Color('column:N'),
    row='sample:N'
)

column_colors = alt.Color('column:N',
    scale=alt.Scale(
        domain=["PGC", "RPLC"],
        range=["#D56E3B", "#58728C"]
    ))
test = d.reset_index()
alt.Chart(test).transform_calculate(
    new_rt = alt.datum.RT + 18.3
).transform_density(
    'RT',
    as_=['RT', 'density'],
    groupby=['sample', 'column'],
).mark_area(
    opacity=0.6
).encode(
    x=alt.X('RT:Q', title='Retention Time (min)',
        scale=alt.Scale(
            domain=[0,100]
        ),
        axis=co.alt_axis),
    y=alt.Y('density:Q', title='density',
        scale=alt.Scale(
            domain=[0,0.05]
        ),
        axis=co.alt_axis),
    color=column_colors,
    column='sample:N'
).properties(
    height=0.9//0.014,
    width=3.65//0.014
)

In [ ]:
def get_valid_counts(dataframe, column, needed):
    counts = dataframe[column].value_counts()
    valid = counts[counts.values==needed].keys()
    kept = dataframe[dataframe[column].isin(valid)]
    return kept

In [ ]:
d = df[(df["Proteotypic"]==1)&(df["Protein.Q.Value"]<=0.01)]
grouped = d.groupby(['sample', 'column', 'Modified.Sequence']).mean()
grouped = grouped.reset_index(['column', 'Modified.Sequence'])

rt_comp = pd.DataFrame()

for s in ["T10", "MT10"]:
    g = grouped.loc[s, :]
    kept = get_valid_counts(g, 'Modified.Sequence', 2).reset_index()
    kept = kept.sort_values('Modified.Sequence').iloc[:750, :]
    rt_comp = pd.concat([rt_comp, kept]).reset_index(drop=True)


rt_comp.loc[:, 'sequence'] = rt_comp["Modified.Sequence"]

# test.loc[:, 'proteins'] = test["Modified.Sequence"]
# counts = test.proteins.value_counts()
# valid = counts[counts.values==4].keys()
# test = test[test.proteins.isin(valid)]

dots = alt.Chart(rt_comp, width=150).mark_circle().encode(
    x=alt.X('column:N', title='', sort=['RPLC', 'PGC'],
        axis=co.alt_axis),
    y=alt.Y('RT:Q', title='RT difference',
        axis=co.alt_axis),
    color=column_colors,
    detail='sequence'
)

lines = alt.Chart(rt_comp, width=150).mark_line(
    color='#888888',
    opacity=0.2
).encode(
    x=alt.X('column:N', title='', sort=['RPLC', 'PGC']),
    y=alt.Y('RT:Q', title='RT difference'),
    # color=column_colors,
    detail='sequence'
).properties(
    height=0.9//0.014,
    width=3.65//0.014
)

alt.layer(lines, dots).facet(
    column='sample:N'
)

In [ ]:
t = test[test['sample']=='T10']
counts = t["Protein.Ids"].value_counts()
valid = counts[counts.values==2].keys()
t = t[t["Protein.Ids"].isin(valid)]
# t = t.sample(frac=0.01)
t = t.sort_values("Protein.Ids").iloc[:50, :]

col_map = dict(zip(["RPLC", 'PGC'], [0, 1]))
t.loc[:, 'col_idx'] = t.column.map(col_map)
t.loc[:, 'proteins'] = t["Protein.Ids"]
alt.Chart(t).mark_line().encode(
    x='col_idx:Q',
    y='RT:Q',
    detail='proteins:N'
)
# t[['sample', 'column', 'Protein.Ids', 'RT']].sort_values('Protein.Ids')

In [ ]:
columns = [c.lower().replace('.', '_') for c in df.columns]
df.columns = columns 

In [ ]:
kept_proteins = df[(df["proteotypic"]==1)&(df["protein_q_value"]<=0.01)]

In [ ]:
grouped = df.groupby(['sample', 'column', 'tech_rep', 'protein_ids']).mean()
grouped = grouped.reset_index('protein_ids')
grouped = grouped.sort_index()

samples = ["T10", "MT10"]
columns = ["PGC", "RPLC"]
reps = ["run1", "run2"]

rep_frame = pd.DataFrame()

for s in samples:
    for c in columns:
        d = pd.DataFrame()
        for r in reps:
            small = grouped.loc[(s, c, r), :]
            data = small[['protein_ids', 'pg_maxlfq']]
            data.columns = ['protein_ids', f'{r}_lfq']
            data.loc[:, f'{r}_log2'] = np.log2(data[f'{r}_lfq'])
            data = data.reset_index()
            data = data.drop('tech_rep', axis=1)


            if d.empty:
                d = data 
            else:
                d = d.merge(data, left_on=['protein_ids', 'sample', 'column'], right_on=['protein_ids', 'sample', 'column'])
            
            d.loc[:, 'line_x'] = np.linspace(0,40,len(d))
            d.loc[:, 'line_y'] = d.line_x

        x, y = d.run1_log2, d.run2_log2 
        xy = np.vstack((x, y))
        print(s, c, stats.pearsonr(x, y))
        density = stats.gaussian_kde(xy)(xy)
        d.loc[:, 'density'] = density
        
        rep_frame = pd.concat([rep_frame, d])
        rep_frame = rep_frame.reset_index(drop=True)


In [ ]:
rep_frame

In [ ]:
rplc_rep = alt.Chart(rep_frame).mark_circle(
    size=10
).encode(
    x=alt.X('run1_log2:Q', title='Run 1',
        axis=co.alt_axis,
        scale=alt.Scale(
            domain=[10,30],
            clamp=True
        )),
    y=alt.Y('run2_log2:Q', title='Run 2',
        axis=co.alt_axis,
        scale=alt.Scale(
            domain=[10,30],
            clamp=True
        )),
    color=alt.Color('density:Q',
        scale=alt.Scale(
            range=['#C8CBCF', '#4B6075']
        )),
    # column='sample:N',
).properties(
    width=400,
    height=200
).transform_filter(
    alt.datum.column=='RPLC'
)

pgc_rep = alt.Chart(rep_frame).mark_circle(
    size=10
).encode(
    x=alt.X('run1_log2:Q', title='Rep 1, log2(abundance)',
        axis=co.alt_axis,
        scale=alt.Scale(
            domain=[10,30],
            clamp=True
        )),
    y=alt.Y('run2_log2:Q', title='Rep 2, log2(abundance)',
        axis=co.alt_axis,
        scale=alt.Scale(
            domain=[10,30],
            clamp=True
        )),
    color=alt.Color('density:Q',
        scale=alt.Scale(
            range=['#D2CFCE', '#B65627']
        )),
    # column='sample:N',
).properties(
    width=400,
    height=200
).transform_filter(
    alt.datum.column=='PGC'
)

line = alt.Chart(rep_frame).mark_line(
    strokeDash=[5,3],
    color='#888888'
).encode(
    x=alt.X('line_x:Q',
        scale=alt.Scale(
            domain=[10,30],
            clamp=True
        )),
    y=alt.Y('line_y:Q',
        scale=alt.Scale(
            domain=[10,30],
            clamp=True
        ))
)

# (rplc_rep+line & pgc_rep+line).facet(
#     'sample:N'
# ).resolve_scale(
#     color='independent'
# )

# (line+rplc_rep).facet('sample:N')

((line+rplc_rep).facet('sample:N') & (line+pgc_rep).facet('sample:N')).resolve_scale(
    color='independent'
)

In [ ]:
rep_frame.loc[:, 'difference'] = rep_frame.run1_log2 - rep_frame.run2_log2
rep_frame.loc[:, 'percent_diff'] = abs(rep_frame.difference) / (rep_frame.run1_log2 + rep_frame.run2_log2) / 2 * 100


box = alt.Chart(rep_frame, width=50).mark_boxplot(
    size=40,
    # opacity=0.4
).encode(
    # x='jitter:Q',
    y=alt.Y('percent_diff:Q', title=r"% difference",
        axis=co.alt_axis),
    color=column_colors,
    column='sample:N',
    row='column:N'
).transform_calculate(
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
)

dens = alt.Chart(rep_frame).mark_area(
    opacity=0.5
).transform_density(
    'percent_diff',
    as_=['percent_diff', 'density'],
    groupby=['sample', 'column']
).encode(
    x=alt.X('percent_diff:Q',
        axis=co.alt_axis,
        scale=alt.Scale(
            domain=[0,2],
            clamp=True
        )),
    y=alt.Y('density:Q', title="density",
        axis=co.alt_axis),
    color=column_colors,
    column='sample:N',
    row='column:N'
)
box | dens

In [ ]:
import seaborn as sns 
# f, ax = plt.subplots()
# sns.violinplot(data=rep_frame[rep_frame.column=='PGC'], x='sample', y='percent_diff', height=4, aspect=0.5)
# sns.despine(offset=10, trim=False)
# plt.ylim(-1, 12)
# sns.violinplot(data=rep_frame[rep_frame.column=='RPLC'], x='sample', y='percent_diff')
g = sns.FacetGrid(rep_frame, col='column', height=4, aspect=0.5)
g.map(sns.violinplot, 'sample', 'percent_diff')
sns.despine(offset=10, trim=True)
plt.savefig('./figures/DIA_percent_difference.svg')

In [ ]:
grouped = kept_proteins.groupby(['sample', 'column', 'protein_ids', 'modified_sequence']).mean()
grouped = grouped.reset_index()

counts = grouped.modified_sequence.value_counts()
valid = counts[counts.values==4].keys()
grouped = grouped[grouped.modified_sequence.isin(valid)]
display(grouped[grouped.protein_ids=='P16104'])


grouped = grouped.groupby(['sample', 'column', 'protein_ids']).sum()
grouped

samples = ["MT10", "T10"]
columns = ["PGC", "RPLC"]

protein_comparison = pd.DataFrame()

for s in samples:
    d = pd.DataFrame()
    for c in columns:
        small = grouped.loc[(s, c), :]
        data = small[['ms1_translated']]
        data.columns = [f'{c}_ms1_translated']
        data.loc[:, f'{c}_log2'] = np.log2(data[f'{c}_ms1_translated'])
        # data.loc[:, 'column'] = c
        data.loc[:, 'sample'] = s
        data.reset_index(inplace=True)
        
        if d.empty:
            d = data 
        else:
            d = d.merge(data, left_on=['protein_ids', 'sample'], right_on=['protein_ids', 'sample'])

        d.loc[:, 'line_x'] = np.linspace(0,40,len(d))
        d.loc[:, 'line_y'] = np.linspace(0,40,len(d))

    protein_comparison = pd.concat([protein_comparison, d])
    protein_comparison = protein_comparison.reset_index(drop=True)

In [ ]:
grouped = kept_proteins.groupby(['sample', 'column', 'protein_ids', 'modified_sequence']).mean()
grouped = grouped.reset_index()

grouped.loc[:, 'prot_seq'] = grouped.apply(lambda x: f'{x.protein_ids}_{x.modified_sequence}', axis=1)

counts = grouped.prot_seq.value_counts()
valid = counts[counts.values==4].keys()
grouped = grouped[grouped.prot_seq.isin(valid)]

grouped = grouped.groupby(['sample', 'column', 'protein_ids']).sum()
grouped

samples = ["MT10", "T10"]
columns = ["PGC", "RPLC"]

protein_comparison = pd.DataFrame()

for s in samples:
    d = pd.DataFrame()
    for c in columns:
        small = grouped.loc[(s, c), :]
        data = small[['ms1_translated']]
        data.columns = [f'{c}_ms1_translated']
        data.loc[:, f'{c}_log2'] = np.log2(data[f'{c}_ms1_translated'])
        # data.loc[:, 'column'] = c
        data.loc[:, 'sample'] = s
        data.reset_index(inplace=True)
        
        if d.empty:
            d = data 
        else:
            d = d.merge(data, left_on=['protein_ids', 'sample'], right_on=['protein_ids', 'sample'])

        d.loc[:, 'line_x'] = np.linspace(0,40,len(d))
        d.loc[:, 'line_y'] = np.linspace(0,40,len(d))
    d.replace([np.inf, -np.inf], np.nan, inplace=True)
    d = d.dropna()
    x, y = d.PGC_log2, d.RPLC_log2
    print(stats.pearsonr(x, y))
    xy = np.vstack((x,y))
    density = stats.gaussian_kde(xy)(xy)

    d.loc[:, 'density'] = density

    protein_comparison = pd.concat([protein_comparison, d])
    protein_comparison = protein_comparison.reset_index(drop=True)

In [ ]:
protein_comparison.protein_ids.unique().shape

In [ ]:
line = alt.Chart(protein_comparison).mark_line(
    strokeDash=[5,3],
    color='#888888'
).encode(
    x=alt.X('line_x',
        scale=alt.Scale(
            domain=[15,40],
            clamp=True
        )),
    y=alt.Y('line_y',
        scale=alt.Scale(
            domain=[15,40],
            clamp=True
        ))
)

dots = alt.Chart(protein_comparison).mark_circle(
    color='#7f7f7f',
    size=10,
    # opacity=0.5
).encode(
    x=alt.X('PGC_log2:Q', title='PGC',
        axis=co.alt_axis,
        scale=alt.Scale(
            domain=[15,40],
            clamp=True
        )),
    y=alt.Y('RPLC_log2:Q', title='RPLC',
        axis=co.alt_axis,
        scale=alt.Scale(
            domain=[15,40],
            clamp=True
        )),
    # color=alt.Color('density:Q', legend=None,
    #     scale=alt.Scale(
    #         scheme='greys', 
    #         reverse=True
    #         # range=['#4B6075', '#A1A1A1', '#B65627']
    #     )),
).properties(
    width=300,
    height=200
)

pre_dot_layer = alt.layer(line, dots).facet(
    'sample:N'
).resolve_scale(
    color='independent'
)

box_1 = alt.Chart(protein_comparison, width=150).mark_boxplot(
    color='#7f7f7f',
    size=65
).encode(
    x=alt.X('sample:N',
        axis=co.alt_axis),
    y=alt.Y('difference:Q',
        axis=co.alt_axis)
).transform_calculate(
    difference=alt.datum.RPLC_log2 - alt.datum.PGC_log2
)

# dot_layer | box_1
pre_dot_layer

In [ ]:
taken_columns = ['sample', 'protein_ids', 'ms1_translated']

rplc = kept_proteins[kept_proteins.column=='RPLC']
pgc = kept_proteins[kept_proteins.column=='PGC']
pgc = pgc[~pgc.modified_sequence.isin(rplc.modified_sequence)]


rplc = rplc.groupby(['sample', 'protein_ids'], as_index=False).sum()
left = rplc[taken_columns]

addition = left.copy()
middle = addition[taken_columns]

pgc = pgc.groupby(['sample', 'protein_ids'], as_index=False).sum()
right = pgc[taken_columns]

a = addition.set_index(['sample', 'protein_ids'])
r = right.set_index(['sample', 'protein_ids'])
pgc_sum = a + r
# pgc_sum = pgc_sum.reset_index()

a+r

In [ ]:
left.columns=['sample', 'protein_ids', 'rplc_ms1']
right.columns=['sample', 'protein_ids', 'pgc_ms1']
m = left.merge(right, on=['sample', 'protein_ids'], how='outer').fillna(0)
m.loc[:, 'new_sum'] = (m.rplc_ms1 + m.pgc_ms1)
m.loc[:, 'log_sum'] = np.log2(m.new_sum)

m.loc[:, 'log_rplc'] = np.log2(m.rplc_ms1)
m.loc[:, 'log_pgc'] = np.log2(m.pgc_ms1)

m.loc[:, 'ratio'] = m.new_sum / m.rplc_ms1
m.loc[:, 'log2_sum'] = np.log10(m.new_sum)
alt.Chart(m[(m.rplc_ms1!=0)&(m.pgc_ms1!=0)]).mark_circle(
    size=5
).encode(
    x=alt.X('ratio:Q', title='Ratio',
        scale=alt.Scale(
            domain=[0,2],
            clamp=True)
        ),
    y=alt.Y('log2_sum:Q', title='Log2(sum)'),
    column='sample:N'
)
alt.Chart(m).mark_circle(
    size=5
).encode(
    x=alt.X('log_rplc:Q'),
    y=alt.Y('log_sum:Q'),
    column='sample:N'
)

In [ ]:
left.columns=['sample', 'protein_ids', 'rplc_ms1']
right.columns=['sample', 'protein_ids', 'pgc_ms1']
m = left.merge(right, on=['sample', 'protein_ids'], how='outer').fillna(0)
m.loc[:, 'new_sum'] = m[['rplc_ms1', 'pgc_ms1']].sum(axis=1)
m.loc[:, 'log_sum'] = np.log2(m.new_sum)
m.loc[:, 'log_rplc'] = np.log2(m.rplc_ms1)
m.loc[:, 'log_pgc'] = np.log2(m.pgc_ms1)
m.loc[:, 'ratio'] = m.new_sum / m.rplc_ms1

kept = m[(m.rplc_ms1!=0)&(m.pgc_ms1!=0)]

for sample in ["MT10", "T10"]:
    small = kept[kept['sample'] == sample]
    kept.loc[small.index, 'line_x'] = np.linspace(0, 40, len(small))
    kept.loc[small.index, 'line_y'] = np.linspace(0, 40, len(small))

    x, y = small.log_pgc, small.log_rplc
    print(stats.pearsonr(x, y))
    xy = np.vstack((x, y))
    density = stats.gaussian_kde(xy)(xy)

    kept.loc[small.index, 'density'] = density


x, y = kept.ratio, kept.rplc_ms1
xy = np.vstack((x, y))
dens = stats.gaussian_kde(xy)(xy)

kept.loc[:, 'density'] = dens


alt.Chart(kept[kept.ratio<=5]).mark_circle(
    color='#403f3f',
    size=20
).encode(
    x=alt.X('ratio:Q', title='Ratio',
        scale=alt.Scale(
            domain=[0,5],
            clamp=True)
        ),
    y=alt.Y('log_sum:Q', title='Log2(sum)'),
    column='sample:N',
    # color=alt.Color('density:Q')
)

dots = alt.Chart(kept).mark_circle(
    color='#7f7f7f',
    size=10
).encode(
    x=alt.X('log_rplc:Q',
        scale=alt.Scale(
            domain=[15,40],
            clamp=True
        ),
        axis=co.alt_axis),
    y=alt.Y('log_sum:Q',
        scale=alt.Scale(
            domain=[15,40],
            clamp=True
        ),
        axis=co.alt_axis),
    # column='sample:N'
).properties(
    width=300,
    height=200
)

line = alt.Chart(kept).mark_line(
    strokeDash=[5,3],
    color='#888888'
).encode(
    x=alt.X('line_x',
        scale=alt.Scale(
            domain=[15,40],
            clamp=True
        )),
    y=alt.Y('line_y',
        scale=alt.Scale(
            domain=[15,40],
            clamp=True
        ))
)
post_dot_layer = alt.layer(line, dots).facet('sample:N')

post_dot_layer

In [ ]:
box = alt.Chart(kept, width=150).mark_boxplot(
    color='#7f7f7f',
    size=65
).encode(
    x=alt.X('sample:N',
        axis=co.alt_axis),
    y=alt.Y('difference:Q',
        axis=co.alt_axis)
).transform_calculate(
    difference=alt.datum.log_rplc - alt.datum.log_pgc
)
(box_1 | box).resolve_scale(
    y='shared'
)

In [ ]:
alt.Chart(kept[kept.ratio<=5]).mark_circle(
    # size=20
).encode(
    x=alt.X('ratio:Q', title='Ratio'),
        
    y=alt.Y('log_sum:Q', title='Log2(sum)'),
    column='sample:N',
    # color=alt.Color('density:Q')
)

In [ ]:
kept